In [6]:
from torchreid import models
models.show_avai_models()

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [7]:
# Get bounding boxes from one picture
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("models/yolov3-tiny.weights", "models/yolov3-tiny.cfg")
classes = []
with open("models/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

    
def detect_person(image):
    # image must be cv2.imread
    # Get image dimensions
    height, width, _ = image.shape

    # Preprocess image
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Get output layer names
    output_layers = net.getUnconnectedOutLayersNames()

    # Forward pass
    detections = net.forward(output_layers)

    # List to store bounding boxes
    bounding_boxes = []

    # Loop over detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # Class ID 0 is 'person'
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                w = int(obj[2] * width)
                h = int(obj[3] * height)

                # Bounding box coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                bounding_boxes.append((x, y, x + w, y + h))

    return bounding_boxes

In [8]:
# Example usage:
image_path = "img1.jpg"
image = cv2.imread(image_path)
# Get bounding boxes
boxes = detect_person(image)
print(boxes)

# Draw bounding boxes on the image
for box in boxes:
    x1, y1, x2, y2 = box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
# Display the result
cv2.namedWindow('Person Detection', cv2.WINDOW_NORMAL)
cv2.imshow("Person Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[(368, 346, 577, 709), (112, 393, 322, 714), (591, 402, 810, 706)]


In [ ]:
# import cv2
# import numpy as np

# def detect_person(image_path):
#     # Đọc ảnh từ đường dẫn
#     image = cv2.imread(image_path)

#     # Đọc pre-trained YOLO model
#     net = cv2.dnn.readNet('models/yolov3-tiny.weights', 'models/yolov3-tiny.cfg')

#     # Load các tên lớp (coco.names) và các cấu hình của mô hình (yolov3.cfg)
#     with open('models/coco.names', 'r') as f:
#         classes = f.read().strip().split('\n')

#     # Chuyển đổi ảnh thành blob để sử dụng với mô hình YOLO
#     blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
#     net.setInput(blob)

#     # Lấy các tên lớp và các điểm tương ứng từ mô hình
#     layer_names = net.getUnconnectedOutLayersNames()
#     outputs = net.forward(layer_names)

#     # Xác định các bounding box, điểm confidence và lớp dự đoán
#     boxes = []
#     confidences = []
#     class_ids = []

#     for output in outputs:
#         for detection in output:
#             scores = detection[5:]
#             class_id = np.argmax(scores)
#             confidence = scores[class_id]

#             if confidence > 0.5 and class_id == classes.index('person'):
#                 # Lấy tọa độ của bounding box
#                 box = detection[0:4] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
#                 (center_x, center_y, width, height) = box.astype('int')

#                 # Chuyển đổi tọa độ về góc trên trái của bounding box
#                 x = int(center_x - (width / 2))
#                 y = int(center_y - (height / 2))

#                 boxes.append([x, y, int(width), int(height)])
#                 confidences.append(float(confidence))
#                 class_ids.append(class_id)

#     # Áp dụng non-maximum suppression để loại bỏ các bounding box trùng lặp
#     indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

#     # Tạo danh sách bounding boxes cuối cùng
#     final_boxes = []
#     for i in indices:
#         i = i[0]
#         final_boxes.append(boxes[i])

#     return final_boxes

In [ ]:
import cv2
from torchvision.transforms import functional as F
from PIL import Image
import torchreid
import torch
from scipy.spatial.distance import cosine

# Tạo một mô hình ReID
model = torchreid.models.build_model(
    name='resnet152',
    num_classes=1000,
    pretrained=True
)

def preprocess_image(image_path):
    # Đọc ảnh và chuyển đổi sang định dạng PyTorch
    image = cv2.imread(image_path)
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    image = F.to_tensor(F.resize(image, (256, 128))).unsqueeze(0)
    return image

# def preprocess_image(image):
#     # Đọc ảnh và chuyển đổi sang định dạng PyTorch
#     image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#     image = F.to_tensor(F.resize(image, (256, 128))).unsqueeze(0)
#     return image

In [ ]:
# Ảnh 1 và 2 là hai ảnh của cùng một nhóm người chụp từ góc khác nhau
image1 = preprocess_image('0.jpg')
image2 = preprocess_image('1.jpg')

In [ ]:
print(image1.shape)
print(type(image1))

In [ ]:
bounding_boxes1 = detect_person(cv2.imread('0.jpg'))
bounding_boxes2 = detect_person(cv2.imread('1.jpg'))
print(bounding_boxes1)
print(bounding_boxes2)

In [ ]:
features1 = []
# Loop over bounding boxes
for bbox in bounding_boxes1:
    x1, y1, x2, y2 = bbox
    roi_image = image1[:, y1:y2, x1:x2]

    # Resize the ROI to the required input size of the model
    resized_roi = F.resize(roi_image, (128, 256))

    # Convert resized_roi to PyTorch tensor
    input_tensor = resized_roi.unsqueeze(0).float()

    # Trích xuất đặc trưng từ ROI
    object_feature = model(input_tensor)['global_feat'].detach().numpy()
    features1.append(object_feature)

features2 = []
# Loop over bounding boxes
for bbox in bounding_boxes2:
    x1, y1, x2, y2 = bbox
    roi_image = image2[:, y1:y2, x1:x2]

    # Resize the ROI to the required input size of the model
    resized_roi = F.resize(roi_image, (128, 256))

    # Convert resized_roi to PyTorch tensor
    input_tensor = resized_roi.unsqueeze(0).float()

    # Trích xuất đặc trưng từ ROI
    object_feature = model(input_tensor)['global_feat'].detach().numpy()
    features2.append(object_feature)

# features1 và features2 là danh sách các đặc trưng của từng đối tượng trong ảnh 1 và 2
print(len(features1), len(features2))

# Tính cosine similarity giữa hai vector đặc trưng
similarity = 1 - cosine(features1[0], features2[0])

# Nếu similarity lớn hơn một ngưỡng nào đó, có thể coi đó là cùng một người
threshold = 0.8
if similarity > threshold:
    print("Cùng một người!")
else:
    print("Không phải cùng một người.")


In [ ]:
# %pip install gdown tensorboard

In [ ]:
# Kiểm tra kích thước của image1
print("image1 Shape:", image1.shape)

for bbox in bounding_boxes1:
    x1, y1, x2, y2 = bbox

    # Kiểm tra giá trị của y1, y2, x1, x2
    print("y1:", y1)
    print("y2:", y2)
    print("x1:", x1)
    print("x2:", x2)

    # Truy cập phần của image1
    roi_image = image1[:, y1:y2, x1:x2]

    # Kiểm tra giá trị và kích thước của roi_image
    print("roi_image Value:", roi_image)
    print("roi_image Shape:", roi_image.shape)

    # Resize the ROI to the required input size of the model
    resized_roi = F.resize(roi_image, (128, 256))

    # Kiểm tra giá trị và kích thước của resized_roi
    print("resized_roi Value:", resized_roi)
    print("resized_roi Shape:", resized_roi.shape)

    # Convert resized_roi to PyTorch tensor
    input_tensor = resized_roi.unsqueeze(0).float()

    # Kiểm tra giá trị và kích thước của input_tensor
    print("input_tensor Value:", input_tensor)
    print("input_tensor Shape:", input_tensor.shape)

    # ... (các dòng mã khác)

